In [16]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import xlrd
from tqdm import tqdm

In [33]:
forders = os.listdir('./data/달러환율데이터/')
print(forders)

['2021년10월', '2021년11월', '2021년12월', '2021년8월', '2021년9월', '2022년10월', '2022년11월', '2022년12월', '2022년1월', '2022년2월', '2022년3월', '2022년4월', '2022년5월', '2022년6월', '2022년7월', '2022년8월', '2022년9월', '2023년1월', '2023년2월', '2023년3월', '2023년4월', '2023년5월', '2023년6월', '2023년7월', 'exchange_rate.csv']


In [18]:
# for forder in forders:
#     for filepath in glob.glob('./data/위안환율데이터/{}/신한은행_환율조회(일자별)_*.xlsx'.format(forder)):
#         print(filepath[-13:-5])
#         # print(filepath)

In [19]:
df = pd.read_excel('./data/위안환율데이터/2021-10/신한은행_환율조회(일자별)_20211001.xlsx',header=5)
df

,고시회차,고시시간,매매기준율,직전대비,송금,Unnamed: 5,현찰,Unnamed: 7,대미환산율
0,NaN,NaN,NaN,NaN,받으실 때,보내실 때,파실 때,사실 때,NaN
1,1.0,08:22:01,183.08,0.00,181.25,184.91,173.93,192.23,0.1545
2,2.0,08:27:32,183.73,증가0.65,181.90,185.56,174.55,192.91,0.1550
3,3.0,09:00:14,183.69,감소0.04,181.86,185.52,174.51,192.87,0.1551
4,4.0,09:01:38,183.74,증가0.05,181.91,185.57,174.56,192.92,0.1551
...,...,...,...,...,...,...,...,...,...
131,131.0,16:58:08,184.39,감소0.04,182.55,186.23,175.18,193.60,0.1555
132,132.0,17:12:29,184.41,증가0.02,182.57,186.25,175.19,193.63,0.1554
133,133.0,17:19:58,184.42,증가0.01,182.58,186.26,175.20,193.64,0.1554
134,134.0,17:32:18,184.40,감소0.02,182.56,186.24,175.18,193.62,0.1554


In [38]:
df = pd.read_excel('./data/달러환율데이터/2021년10월/신한은행_환율조회(일자별)_20211001.xls',header=5)
df.loc[2:,['고시시간','매매기준율','직전대비']]

,고시시간,매매기준율,직전대비
2,17:32:18,"1,186.50",감소0.50
3,17:19:58,"1,187.00",증가0.50
4,17:12:29,"1,186.50",증가0.50
5,16:58:08,"1,186.00",감소0.50
6,16:52:34,"1,186.50",0.00
...,...,...,...
131,09:02:50,"1,184.70",감소0.30
132,09:01:38,"1,185.00",증가0.30
133,09:00:14,"1,184.70",감소0.30
134,08:27:32,"1,185.00",감소0.20


In [37]:
df['고시시간']

0          고시시간
1      18:00:08
2      17:32:18
3      17:19:58
4      17:12:29
         ...   
131    09:02:50
132    09:01:38
133    09:00:14
134    08:27:32
135    08:22:01
Name: 고시시간, Length: 136, dtype: object

In [41]:
df_all = pd.DataFrame()
open_dates = []
for forder in tqdm(forders):
    for filepath in glob.glob('./data/달러환율데이터/{}/신한은행_환율조회(일자별)_*.xls'.format(forder)):
        # 파일 날짜
        f_date = filepath[-12:-4]
        # 영업일자 목록
        open_dates.append(f_date)
        # 파일 불러오기 + 파일날짜로 변수명 지정
        globals()['df_{}'.format(f_date)] = pd.read_excel(filepath, header = 5)
        
        # 필요한 컬럼만 
        globals()['df_{}'.format(f_date)] = globals()['df_{}'.format(f_date)].loc[2:,['고시시간','매매기준율','직전대비']]
        globals()['df_{}'.format(f_date)].reset_index(inplace=True, drop=True)
        # 날짜 컬럼 추가
        globals()['df_{}'.format(f_date)].insert(0,'날짜',f_date)


        # 빈 시간 채우기

        # 정상 Datetime row (0900~1520)
        timelist = pd.date_range('09:00','15:20',freq='1min').time.astype('str')
        # '시간:분'
        for i in range(len(timelist)):
            timelist[i] = timelist[i][:-3]
    

        # 수정할 날짜 데이터
        
        # '시간:분'
        globals()['df_{}'.format(f_date)]['고시시간'] = globals()['df_{}'.format(f_date)]['고시시간'].apply(lambda _ : _[:-3])
        dftime = globals()['df_{}'.format(f_date)]['고시시간']

        # 9시00분~15시20분 외 데이터 자르기
        i=0
        for time_ in dftime :
            time_ = datetime.strptime(time_, '%H:%M').time()
            if (time_ > datetime.strptime('15:20','%H:%M').time()) or (time_ < datetime.strptime('09:00','%H:%M').time()) :
                globals()['df_{}'.format(f_date)].drop(index=[i],axis=1,inplace=True)
                
                i+=1
            else:
                # print(time_)
                i+=1


        # 빈시간 row :정상 - 수정
        abtimes =  list(set(timelist)-set(dftime))

        testdf = pd.DataFrame(columns=globals()['df_{}'.format(f_date)].columns)
        testdf['고시시간'] = abtimes
        globals()['df_{}'.format(f_date)] = pd.concat([globals()['df_{}'.format(f_date)],testdf])

        globals()['df_{}'.format(f_date)].sort_values(by='고시시간', ascending=True, inplace=True)



        # 중복 시간 제거
        globals()['df_{}'.format(f_date)].drop_duplicates(['고시시간'], keep='first', inplace=True, ignore_index=True)

        globals()['df_{}'.format(f_date)]['날짜'].fillna(f_date,inplace=True)
        globals()['df_{}'.format(f_date)]['매매기준율'].ffill(inplace=True)
        globals()['df_{}'.format(f_date)]['직전대비'].fillna(0, inplace=True)

        # globals()['df_{}'.format(f_date)] = globals()['df_{}'.format(f_date)][globals()['df_{}'.format(f_date)]['고시시간'] != '09:00']
    
        # 날짜 + 시간 = Datetime 컬럼  추가
        globals()['df_{}'.format(f_date)].insert(0,'Datetime',f_date + globals()['df_{}'.format(f_date)]['고시시간'])
        globals()['df_{}'.format(f_date)]['Datetime'] = globals()['df_{}'.format(f_date)]['Datetime'].apply(lambda _ : datetime.strptime(_,'%Y%m%d%H:%M'))
        # 시간순 정렬
        globals()['df_{}'.format(f_date)].sort_values('Datetime',inplace=True,ignore_index=True)
        

        # globals()['df_{}'.format(f_date)] = globals()['df_{}'.format(f_date)][['Datetime','매매기준율','직전대비']]

        df_all = pd.concat([df_all, globals()['df_{}'.format(f_date)]])


  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:32<00:00,  1.29s/it]


In [42]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185547 entries, 0 to 380
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Datetime  185547 non-null  datetime64[ns]
 1   날짜        185547 non-null  object        
 2   고시시간      185547 non-null  object        
 3   매매기준율     185456 non-null  object        
 4   직전대비      185547 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 8.5+ MB


In [43]:
df_all.reset_index(inplace=True, drop=True)

In [44]:
df_all

,Datetime,날짜,고시시간,매매기준율,직전대비
0,2021-10-01 09:00:00,20211001,09:00,"1,184.70",감소0.30
1,2021-10-01 09:01:00,20211001,09:01,"1,185.00",증가0.30
2,2021-10-01 09:02:00,20211001,09:02,"1,184.70",감소0.30
3,2021-10-01 09:03:00,20211001,09:03,"1,184.70",0
4,2021-10-01 09:04:00,20211001,09:04,"1,184.60",감소0.10
...,...,...,...,...,...
185542,2023-07-20 15:16:00,20230720,15:16,"1,269.00",감소0.30
185543,2023-07-20 15:17:00,20230720,15:17,"1,269.40",증가0.20
185544,2023-07-20 15:18:00,20230720,15:18,"1,269.90",감소0.20
185545,2023-07-20 15:19:00,20230720,15:19,"1,269.90",0.00


In [45]:
# 9시00분~15시20분 외 데이터 자르기
i=0
for time_ in df_all['고시시간'] :
    time_ = datetime.strptime(time_, '%H:%M').time()
    if (time_ > datetime.strptime('15:20','%H:%M').time()) or (time_ < datetime.strptime('09:00','%H:%M').time()) :
        df_all.drop(index=[i],axis=1,inplace=True)
        
        i+=1
    else:
        # print(time_)
        i+=1

In [46]:
df_all['매매기준율'].ffill(inplace=True)

In [47]:
df_all[df_all['매매기준율'].isnull()]

,Datetime,날짜,고시시간,매매기준율,직전대비


In [48]:
df_all = df_all[df_all['고시시간'] != '09:00']

In [49]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185060 entries, 1 to 185546
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Datetime  185060 non-null  datetime64[ns]
 1   날짜        185060 non-null  object        
 2   고시시간      185060 non-null  object        
 3   매매기준율     185060 non-null  object        
 4   직전대비      185060 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 8.5+ MB


In [50]:
df_all.sort_values(by='Datetime',inplace=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11848\3128194535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all.sort_values(by='Datetime',inplace=True)


In [31]:
type(df_all['날짜'].unique()[0])

str

In [32]:
df_all.to_csv('./data/환율데이터/exchange_rate_CNY.csv',index=False)

In [51]:
df = pd.read_csv('./data/달러환율데이터/exchange_rate.csv')

In [93]:
df = df.iloc[:9500,:]

In [94]:
df_all

,Datetime,날짜,고시시간,매매기준율,직전대비
24385,2021-08-02 09:01:00,20210802,09:01,"1,151.50",감소0.40
24386,2021-08-02 09:02:00,20210802,09:02,"1,151.60",증가0.30
24387,2021-08-02 09:03:00,20210802,09:03,"1,151.30",감소0.30
24388,2021-08-02 09:04:00,20210802,09:04,"1,151.20",감소0.10
24389,2021-08-02 09:05:00,20210802,09:05,"1,151.40",증가0.20
...,...,...,...,...,...
185542,2023-07-20 15:16:00,20230720,15:16,"1,269.00",감소0.30
185543,2023-07-20 15:17:00,20230720,15:17,"1,269.40",증가0.20
185544,2023-07-20 15:18:00,20230720,15:18,"1,269.90",감소0.20
185545,2023-07-20 15:19:00,20230720,15:19,"1,269.90",0.00


In [95]:
df_All = pd.concat([df,df_all])

In [96]:
df_All

,Datetime,날짜,고시시간,매매기준율,직전대비
0,2021-06-28 09:01:00,20210628,09:01,"1,127.90",-0.1
1,2021-06-28 09:02:00,20210628,09:02,"1,127.90",0.0
2,2021-06-28 09:03:00,20210628,09:03,"1,127.90",0.0
3,2021-06-28 09:04:00,20210628,09:04,"1,127.90",0.0
4,2021-06-28 09:05:00,20210628,09:05,"1,128.50",0.6
...,...,...,...,...,...
185542,2023-07-20 15:16:00,20230720,15:16,"1,269.00",감소0.30
185543,2023-07-20 15:17:00,20230720,15:17,"1,269.40",증가0.20
185544,2023-07-20 15:18:00,20230720,15:18,"1,269.90",감소0.20
185545,2023-07-20 15:19:00,20230720,15:19,"1,269.90",0.00


In [97]:
df_All.reset_index(inplace=True, drop=True)

In [98]:
df_All['날짜'][0]

20210628

In [99]:
# 폐장일 데이터 삭제
i=0
for date_ in df_All['날짜']:
    if date_ == '20211231':
        df_All.drop(index=i,axis=1,inplace=True)
        i+=1
    else:
        i+=1

In [101]:
df_All.to_csv('./data/환율데이터/exchange_rate_USD.csv',index=False)

In [100]:
df_All.info()

<class 'pandas.core.frame.DataFrame'>
Index: 194180 entries, 0 to 194559
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Datetime  194180 non-null  object
 1   날짜        194180 non-null  object
 2   고시시간      194180 non-null  object
 3   매매기준율     194180 non-null  object
 4   직전대비      194180 non-null  object
dtypes: object(5)
memory usage: 8.9+ MB
